In [1]:
import sys
import os
fname = r'C:\Users\agmontesb\Documents\GitHub\excel\tests\test_base_workbook.py'
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), '..')))

In [2]:
import pytest
import openpyxl as px
import pandas as pd
import itertools
import collections

from excel_workbook import (
    ExcelWorkbook, ExcelTable, 
    cell_address, cell_pattern, 
    data_in_range, tbl_address, rgn_pattern,
    XlErrors, TABLE_DATA_MAP, EMPTY_CELL, 
    tbl_pattern, rgn_pattern
    )



In [3]:
def base_workbook(case):
    # Create a base workbook for testing
    filename = r"C:\Users\agmontesb\Downloads\excel_module_test.xlsx"
    wb = px.load_workbook(filename)

    excel_wb = ExcelWorkbook('excel_module_test')

    ws_name = "No links, No parameters"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["G4:I9", "F3:I9"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh1_tbl1 = ExcelTable(wsheet, 'sh1_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh1_tbl1 = sh1_tbl1.minimun_table()

    # Tabla 2
    ws_range = ["G11:H15", "F12:H15"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh1_tbl2 = ExcelTable(wsheet, 'sh1_tbl2', ws_range, fmls, values, recalc=True)
    m_sh1_tbl2 = sh1_tbl2.minimun_table()

    ws_name = "Parameters and inner links"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["F4:H9", "E3:H9"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh2_tbl1 = ExcelTable(wsheet, 'sh2_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh2_tbl1 = sh2_tbl1.minimun_table()

    # Tabla 2
    ws_range = ["F13:H17", "E12:H17"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh2_tbl2 = ExcelTable(wsheet, 'sh2_tbl2', ws_range, fmls, values, recalc=True)
    # m_sh2_tbl2 = sh2_tbl2.minimun_table()

    ws_name = "Outer links, outer parameter"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["F3:H8", "E2:H8"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh3_tbl1 = ExcelTable(wsheet, 'sh3_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh3_tbl1 = sh3_tbl1.minimun_table()

    wb.close()
    return excel_wb


In [4]:
wb = base_workbook(1)

In [5]:
wb.sheets[1]

col,E,F,G,H
row,,,,
2,,,0,
3,tabla 1:,,,
4,l1,700,250,950
5,l2,1200,230,1430
6,l3,340,100,440
7,l4,128,85,213
8,l5,45,18,63
9,,2413,683,3096
10,,,,


In [6]:
ws = wb.sheets[1]
tbl = ws.tables[0]
df = tbl.data

In [7]:
from excel_workbook import CIRCULAR_REF

In [8]:
set_up = dict(G3='=10+F3', H3='=10+G3', F3='10+H3')
values = dict(G6='=G3+100')
answer = {'F3': '0', 'G3': '', 'H3': ''}


In [9]:
values = values or {}
tbl = wb['Parameters and inner links']['sh2_tbl1']
tbl.set_records(set_up, field='fml')
tbl.recalculate(recalc=True)
if values:
    tbl.set_records(values, field='fml')
    tbl.recalculate(recalc=True)
assert (tbl.data.loc[answer.keys()].value.apply(str) == pd.Series(answer)).all()
keys = list(set_up.keys() | values.keys())
codes = tbl.data.loc[keys].code.tolist()
dependents = tbl.parent.all_dependents({tbl: set(codes)}, with_links=True)
assert all(
    val == CIRCULAR_REF
    for ftbl, codes in dependents.items()
    if not (df := ftbl.data).empty
    for val in df.loc[df.code.isin(codes)].value
)


In [18]:
(
    wb.sheets[2].tables[0].minimun_table()
    .unstack()
    .to_frame()
    .assign(is_circ=lambda df: df[0] == CIRCULAR_REF)
    .sort_values('is_circ')
)

c:\Users\agmontesb\Documents\GitHub\excel\excel_workbook.py:1875: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = self[excel_slice].fillna(EMPTY_CELL)


0  is_circ
col row              
E   3     l1    False
H   6     45    False
    4    230    False
    3    240    False
G   7     18    False
    6     85    False
    4    230    False
    3    250    False
H   7    -32    False
F   7     50    False
    8    130    False
    5     30    False
    4      0    False
    3     10    False
E   8           False
    7     l5    False
    6     l4    False
    5     l3    False
    4     l2    False
F   6     40    False
G   5    100     True
    8    683     True
H   5     70     True
    8    553     True

In [19]:
wb.sheets[2].tables[0].data

,fml,dependents,res_order,ftype,value,code
cell,,,,,,
"'No links, No parameters'!G4",NaN,{A1},0,#,10,'sheet'!A9
"'No links, No parameters'!G5",NaN,{A4},0,#,20,'sheet'!A11
"'No links, No parameters'!G6",NaN,{A7},0,#,30,'sheet'!A13
"'No links, No parameters'!G7",NaN,{A10},0,#,40,'sheet'!A16
"'No links, No parameters'!G8",NaN,{A13},0,#,50,'sheet'!A18
'Parameters and inner links'!G2,NaN,{A4},0,#,0,'sheet1'!ZG2
'Parameters and inner links'!G4,NaN,{A2},0,#,250,'sheet1'!A9
'Parameters and inner links'!G5,NaN,{A5},0,#,230,'sheet1'!A12
'Parameters and inner links'!G6,NaN,{A8},0,#,100,'sheet1'!A13


In [20]:
wb.sheets[2].tables[0].data.loc["'Parameters and inner links'!G6"].value

In [ ]:
dmy =  [1,2]
dmy.extend({3, 4})
dmy

In [ ]:
keys = dmy.keys()
[key.title for key in keys]

In [ ]:
dmy[list(keys)[1]]

In [37]:
import itertools
from excel_workbook import cell_pattern, rgn_pattern

def is_cell_in_fml(cell, fml):
        code = lambda cell: '{1:0>6s}|{0:_>3s}'.format(*cell_pattern.match(cell).groups()[1:]).split('|')
        if not fml:
            return False
        components = [x.replace('$', '').split(':') for x in set(rgn_pattern.findall(fml))]
        if not any(x == cell for x in itertools.chain(*components)):
            cell_code = code(cell)
            rng_limits = (
                [code(x) for x in pair]
                for pair in components if len(pair) == 2
            )
            # mask = [t[0] <= cell_code <= t[1] for t in rng_limits]
            # print(f'{cell_code=}, {mask=}')
            bflag = any(t[0][0] <= cell_code[0] <= t[1][0] and t[0][1] <= cell_code[1] <= t[1][1] for t in rng_limits)
            return bflag
        return True


In [42]:
is_cell_in_fml('F6', '=+SUM(G4:G8)')

False

In [8]:
values = {
    'I13': '=F13+F15',
    'I14': '=I13+5',
    'I15': '=I14+10',
}
tbl.set_records(values, field='fml')

In [10]:
values = {
    'I13': '=1000+I15',
}
tbl.set_records(values, field='fml')

In [9]:
tbl.recalculate(recalc=True)

In [ ]:
tbl

In [ ]:
tbl.evaluate('=E13&" "&E16')

In [23]:
import functools
reduce_sets1 = lambda items: functools.reduce(lambda t, e: t.union(e) or t, items, set())
reduce_sets2 = lambda items: set(itertools.chain(*items))


In [31]:
s = pd.Series(dict(l1=set([1,2,3]), l2=set([4,5,6]), l3=set(), l4=set([80]), l5=set([22])))
s.isin([set()])

l1    False
l2    False
l3     True
l4    False
l5    False
dtype: bool

In [77]:
dmy ='''tabla2:			
l1	25	100	0
l2	10	25	35
l3	15	38	53
l4	80	2438	2518
	105	2501	2606'''

In [52]:
dmy= '''
tabla 1:			
l1	700	250	950
l2	1200	230	1430
l3	340	100	440
l4	128	85	213
l5	45	18	63
	2413	683	3096
'''

In [13]:
lst = [x.split('\t') for x in dmy.split('\n')] 

In [ ]:
pd.DataFrame(lst)

In [50]:
def excel_df(tbl_str, row_ini, col_ini):
    lst = [x.split('\t') for x in tbl_str.strip().split('\n')]
    ndx1, ndx2 = int(row_ini), int(row_ini) + len(lst)                 #map(int, row_rng.split(':'))
    col1, col2 = ord(col_ini), ord(col_ini) + len(lst[0])                            #map(ord, col_rng.split(':'))
    df = (
        pd.DataFrame(lst, index=pd.RangeIndex(ndx1, ndx2), columns=[chr(x) for x in range(col1, col2)])
        .unstack()
        .to_frame()
        .rename(columns={0:'value'})
        .assign(cell=lambda db: db.index.map(lambda x: x[0] + str(x[1])))
        .set_index('cell')
    )
    return df.value

In [82]:
base = excel_df(dmy, '12', 'E')

In [127]:
snew = '''tabla2:	0	20	30
l1	25	100	0
l2	10	25	35
l3	15	38	53
l4	80	2438	2518
	105	2501	2606
'''

In [128]:
vnew = excel_df(snew, '12', 'E')

In [ ]:
mask = vnew != base
str(vnew[mask].to_dict()).replace('\n', '')

In [106]:
code = '''{'F13': '3',
 'G14': '41',
 'G16': '2416',
 'G17': '2495',
 'H14': '51',
 'H16': '2496',
 'H17': '2600'}'''

In [56]:
cell_address('A5')

('5', 'A')

In [82]:
cells = ['A5', 'B10', 'AA1', 'A3']
cell = cells[-1]
def address_id(cell, nbase=4, max_col=None):
    if max_col is None:
        max_col = sum(nbase ** (k + 1) for k in range(nbase))
    row, col = cell_address(cell)
    t0 = (int(row) - 1) * max_col
    t = col[::-1]
    for k, c in enumerate(t):
        t0 += (ord(c) - ord('A') + 1) * nbase ** k
    return t0
cell, address_id(cell)


('A3', 681)

In [83]:
def id_address(id, nbase=4, max_col=None):
    if max_col is None:
        max_col = sum(nbase ** (k + 1) for k in range(nbase))
    row = id // max_col
    id = id - row * max_col
    row += 1
    answ = [str(row)]
    while id:
        res = id % nbase
        id = id // nbase
        if res == 0:
            res = nbase
            id -=1
        answ.append(chr(ord('A') + res - 1))
    return ''.join(answ[::-1])

id_address(681)



'A3'